In [13]:
#Load a pretrained UNet model and perform pixel-wise segmentation on new images.
#!pip install -U segmentation-models
import os
os.environ["SM_FRAMEWORK"] = "tf.keras" # Force library to use tensorflow.keras

import segmentation_models as sm
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 1. Load Model (e.g., using ResNet backbone)
model = sm.Unet('resnet34', classes=1, activation='sigmoid', encoder_weights='imagenet')

# 2. Prepare Image (Resize to a multiple of 32 for U-Net)
img_raw = load_img("image.jpg", target_size=(256, 256))
img_array = img_to_array(img_raw) / 255.0

# 3. Predict & Display
mask = model.predict(np.expand_dims(img_array, axis=0))[0]
plt.imshow(img_raw)
plt.imshow(mask > 0.5, alpha=0.5) # Overlay mask
plt.show()

Segmentation Models: using `tf.keras` framework.
85521592/85521592 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


FileNotFoundError: [Errno 2] No such file or directory: 'image.jpg'

In [15]:
#Build a CNN that classifies handwritten digits using the MNIST dataset.
import tensorflow as tf
from tensorflow.keras import layers, models

# Load dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixels to [0, 1] and reshape to (28, 28, 1)
x_train = x_train.reshape((-1, 28, 28, 1)).astype("float32") / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype("float32") / 255.0
model = models.Sequential([
    # First Convolutional block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    
    # Second Convolutional block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten and Classify
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # 10 digits (0-9)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train for 5 epochs
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

/Users/harikrishnan/Documents/anaconda3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.9570 - loss: 0.1436 - val_accuracy: 0.9849 - val_loss: 0.0471
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - accuracy: 0.9850 - loss: 0.0481 - val_accuracy: 0.9851 - val_loss: 0.0461
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 37s 20ms/step - accuracy: 0.9888 - loss: 0.0342 - val_accuracy: 0.9896 - val_loss: 0.0317
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - accuracy: 0.9922 - loss: 0.0248 - val_accuracy: 0.9891 - val_loss: 0.0341
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - accuracy: 0.9940 - loss: 0.0192 - val_accuracy: 0.9921 - val_loss: 0.0246


In [16]:
#Write a function that reads an image, resizes it, and normalizes it for inference.
import cv2
import numpy as np

def preprocess_image(image_path, target_size=(224, 224)):
    # 1. Read image (OpenCV reads in BGR by default)
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found at the specified path")
    
    # 2. Convert to RGB and Resize
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
    
    # 3. Normalize to [0, 1] and add batch dimension
    img_array = img.astype('float32') / 255.0
    img_batch = np.expand_dims(img_array, axis=0)
    
    return img_batch

In [21]:
#detect objects YOLO
import torch

# Load the pretrained YOLOv5s model from PyTorch Hub
!pip install -U ultralytics

import ultralytics
print(ultralytics.__version__)

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Path to your image (or a URL)
img = 'path/to/vehicle_image.jpg' 

# Run the model
results = model(img)

  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 11.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 MB 33.0 MB/s  0:00:01 eta 0:00:01
Using cached ultralytics_thop-2.0.18-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ultralytics] [ultralytics]thop]
8.3.249


Using cache found in /Users/harikrishnan/.cache/torch/hub/ultralytics_yolov5_master


ModuleNotFoundError: No module named 'tqdm'

In [22]:
#Use an Artificial Neural Network to predict whether a customer will buy a product (0 = No, 1 = Yes), based on Age and Salary.
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Sample data: [Age, Salary]
X = np.array([[25, 30000], [35, 70000], [45, 120000], [20, 15000], [55, 60000]])
y = np.array([0, 1, 1, 0, 1])

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = tf.keras.models.Sequential([
    # Input Layer + First Hidden Layer
    tf.keras.layers.Dense(units=6, activation='relu', input_dim=2),
    # Second Hidden Layer
    tf.keras.layers.Dense(units=6, activation='relu'),
    # Output Layer (Sigmoid for binary probability)
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_scaled, y, epochs=100, verbose=0)

# Predict for a new customer (Age: 30, Salary: 50,000)
new_data = scaler.transform([[30, 50000]])
prediction = model.predict(new_data)
print(f"Purchase Probability: {prediction[0][0]:.2f}")

/Users/harikrishnan/Documents/anaconda3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Purchase Probability: 0.42
